In [7]:
!pip install simpy
!pip install numpy


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\Lennard\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     ------------ ------------------------- 20.5/60.8 kB 217.9 kB/s eta 0:00:01
     ------------------------- ------------ 41.0/60.8 kB 281.8 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 324.4 kB/s eta 0:00:00
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   ---------------------------------------- 0.1/12.9 MB 3.3 MB/s eta 0:00:04
   - -------------------------------------- 0.4/12.9 MB 4.6 MB/s eta 0:00:03
   -- ------------------------------------- 0.7/12.9 MB 5.5 MB/s eta 0:00:03
   -- ------------------------------------- 0.9/12.9 MB 5.4 MB/s eta 0:00:03
   --- ------------------------------------ 1.2/12.9 MB 5.4 MB/s eta 0:00:03
   ---- ----------------------------------- 1.5/12.9 MB 5.4 MB/s eta 0:00:03
   ----- ---------------------------------- 1.7/12.9 MB 5.5 MB/s eta 0:00:03
   ---


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\Lennard\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
import simpy
import numpy as np

# Define the parameters
class IntersectionSimulation:
    def __init__(self, env, arrival_distributions, crossing_times, row_times):
        self.env = env
        self.arrival_distributions = arrival_distributions  # Vehicle entrance distributions
        self.crossing_times = crossing_times  # Crossing time distribution parameters
        self.row_times = row_times  # Time to move from row i to row i-1
        self.queues = {street: [] for street in arrival_distributions}
        self.light_state = {street: False for street in arrival_distributions}
        self.results = {street: [] for street in arrival_distributions}
        self.vehicle_count = 0  # Counter to number vehicles

        env.process(self.traffic_light_cycle())
        for street in arrival_distributions:
            env.process(self.vehicle_arrival(street))

    def traffic_light_cycle(self):
        """Controls the traffic light system."""
        cycle_order = [('Bakeri1', 30), ('Bakeri2', 25), ('Besat1', 30), ('Besat2', 30)]
        while True:
            for street, green_time in cycle_order:
                self.light_state[street] = True
                if self.env.now < 500:
                    print(f"Time {self.env.now:.2f}: Light turned GREEN for {street}")
                yield self.env.timeout(green_time)
                self.light_state[street] = False

    def vehicle_arrival(self, street):
        """Generates vehicles arriving at the intersection."""
        while True:
            inter_arrival_time = max(0.1, self.arrival_distributions[street]())  # Ensure positive delay
            yield self.env.timeout(inter_arrival_time)
            self.vehicle_count += 1
            vehicle_id = self.vehicle_count
            self.env.process(self.vehicle_cross(street, vehicle_id))

    def vehicle_cross(self, street, vehicle_id):
        """Simulates a vehicle's journey through the intersection."""
        arrival_time = self.env.now
        queue_length = len(self.queues[street])
        wait_time = queue_length * self.row_times[street]

        self.queues[street].append((vehicle_id, arrival_time))

        yield self.env.timeout(wait_time)
        while not self.light_state[street]:
            yield self.env.timeout(1)  # Wait for green light

        crossing_time = max(0.1, self.crossing_times[street]())  # Ensure positive crossing time
        yield self.env.timeout(crossing_time)

        _, start_time = self.queues[street].pop(0)
        cycle_time = self.env.now - start_time
        self.results[street].append(cycle_time)

        if self.env.now < 500:  # Print only for the first 200 time units
            print(f"Time {self.env.now:.2f}: Vehicle {vehicle_id} from {street} crossed after {cycle_time:.2f} seconds")

    def get_results(self):
        return {street: np.mean(times) if times else 0 for street, times in self.results.items()}

# Define scenario parameters
arrival_distributions = {
    'Bakeri1': lambda: max(0.1, -0.5 + 22 * np.random.beta(0.971, 2.04)),
    'Bakeri2': lambda: max(0.1, -0.5 + 41 * np.random.beta(0.968, 3.44)),
    'Besat1': lambda: max(0.1, -0.5 + 23 * np.random.beta(0.634, 1.61)),
    'Besat2': lambda: max(0.1, -0.5 + 24 * np.random.beta(0.963, 1.99))
}

crossing_times = {
    'Bakeri1': lambda: max(0.1, np.random.poisson(8.21)),
    'Bakeri2': lambda: max(0.1, np.random.normal(5.66, 2.08)),
    'Besat1': lambda: max(0.1, 2.5 + min(15, np.random.lognormal(7.65, 6.42))),
    'Besat2': lambda: max(0.1, 2.5 + np.random.gamma(2.82, 3))
}

row_times = {
    'Bakeri1': 2.675,
    'Bakeri2': 2.4666,
    'Besat1': 2.6818,
    'Besat2': 2.3243
}

# Run the simulation
env = simpy.Environment()
sim = IntersectionSimulation(env, arrival_distributions, crossing_times, row_times)
env.run(until=10000)  # Run simulation for 10,000 time units

# Print results
results = sim.get_results()
print("Average vehicle cycle time per street:")
for street, avg_time in results.items():
    print(f"{street}: {avg_time:.2f} seconds")


Time 0.00: Light turned GREEN for Bakeri1
Time 15.43: Vehicle 2 from Bakeri1 crossed after 7.00 seconds
Time 21.39: Vehicle 3 from Bakeri1 crossed after 12.67 seconds
Time 27.30: Vehicle 6 from Bakeri1 crossed after 15.35 seconds
Time 30.00: Light turned GREEN for Bakeri2
Time 35.45: Vehicle 4 from Bakeri2 crossed after 25.68 seconds
Time 35.73: Vehicle 11 from Bakeri2 crossed after 16.87 seconds
Time 38.07: Vehicle 12 from Bakeri1 crossed after 14.68 seconds
Time 38.85: Vehicle 13 from Bakeri1 crossed after 15.35 seconds
Time 45.59: Vehicle 20 from Bakeri2 crossed after 5.23 seconds
Time 50.55: Vehicle 26 from Bakeri2 crossed after 0.62 seconds
Time 55.00: Light turned GREEN for Besat1
Time 60.09: Vehicle 8 from Besat1 crossed after 59.99 seconds
Time 72.54: Vehicle 22 from Besat1 crossed after 59.09 seconds
Time 72.60: Vehicle 1 from Besat1 crossed after 56.47 seconds
Time 72.61: Vehicle 17 from Besat1 crossed after 35.55 seconds
Time 72.99: Vehicle 10 from Besat1 crossed after 30.68